<a href="https://colab.research.google.com/github/aminanss/EEG-Foundation-Challenge/blob/main/EEG_Foundation_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet s3fs mne pandas

import os
import s3fs
import pandas as pd
import json

In [ ]:
# 0) (Make sure you have turned Internet: ON in the Notebook Settings)

# 1) Install required libraries quietly
!pip install --quiet s3fs mne pandas matplotlib

import os
import s3fs
import pandas as pd
import json
import mne
import matplotlib.pyplot as plt

# 2) Initialize an anonymous S3 filesystem
fs = s3fs.S3FileSystem(anon=True)

# 3) Define the S3 prefix path for the dataset
s3_prefix = 'fcp-indi/data/Projects/HBN/BIDS_EEG/cmi_bids_R1'

# 4) List top‐level folders under the prefix
print("Top‐level entries under", s3_prefix)
for entry in fs.ls(s3_prefix, detail=False):
    print("  ", entry)

# 5) Read participants.tsv directly from S3 into a DataFrame
participants_path = os.path.join(s3_prefix, 'participants.tsv')
with fs.open(participants_path, 'rb') as f:
    participants = pd.read_csv(f, sep='\t')
print("\n---- participants.tsv (first rows) ----")
print(participants.head())

# 6) Read dataset_description.json directly from S3
dataset_desc_path = os.path.join(s3_prefix, 'dataset_description.json')
with fs.open(dataset_desc_path, 'rb') as f:
    dataset_description = json.load(f)
print("\n---- dataset_description.json ----")
print(json.dumps(dataset_description, indent=2))

# 7) Find a subject folder (e.g., 'sub-XXXX') under the prefix
subject_dirs = [p for p in fs.ls(s3_prefix, detail=False)
                if p.endswith('/') and 'sub-' in p]
if not subject_dirs:
    raise RuntimeError("No subject folders found under the S3 prefix.")
subject_folder = subject_dirs[0].rstrip('/')
print(f"\nSelected subject folder: {subject_folder}")

# 8) List EEG files inside the subject’s 'eeg' subfolder
eeg_dir = f"{subject_folder}/eeg"
print("\nEEG files available under that subject:")
for fname in fs.ls(eeg_dir, detail=False):
    print("  ", fname)

# 9) Download the BrainVision files (.vhdr, .eeg, .vmrk) locally
local_eeg_dir = '/kaggle/working/eeg_sample'
os.makedirs(local_eeg_dir, exist_ok=True)

bv_files = [f for f in fs.ls(eeg_dir, detail=False)
            if f.endswith(('.vhdr', '.eeg', '.vmrk'))]
print(f"\nDownloading {len(bv_files)} BrainVision files to {local_eeg_dir}")
for remote_file in bv_files:
    local_file = os.path.join(local_eeg_dir, os.path.basename(remote_file))
    fs.get(remote_file